In [1]:
import os.path as osp
import random
# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as data

%matplotlib inline

In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
# 学習、検証の画像データとアノテーションデータへのファイルパスリストを作成する

def make_datapath_list(rootpath):
    """
    データへのパスを格納したリストを作成する。
    
    Parameters
    ----------
    rootpath : str
        データフォルダへのパス
    
    Rerutns
    -------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        データへのパスを格納したリスト
    """
    
    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')  # %を使っているのはl.32などを見ればわかる
    annopath_template = osp.join(rootpath, 'Annotations', '%s.xml')
    
    # 訓練と検証、それぞれのファイルのID(ファイル名)を取得する
    train_id_names = osp.join(rootpath + 'ImageSets/Main/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Main/val.txt')
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
    train_img_list = list()
    train_anno_list = list()
    
    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)
        
    # 検証データの画像ファイルとアノテーションファイルへのパスリストを作成
    val_img_list = list()
    val_anno_list = list()
    
    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [7]:
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# 動作確認
print(train_img_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg


In [8]:
class Anno_xml2list(object):
    """
    1枚の画像に対する「xml形式のアノテーションデータ」を、画像サイズで規格化してからリスト形式に変換する。
    
    Attributes
    ----------
    classes : リスト
        VOCのクラス名を格納したリスト
    """
    
    def __init__(self, classes):
        self.classes = classes
        
    def __call__(self, xml_path, width, height):
        """
        1枚の画像に対する「XML形式のアノテーションデータ」を、画像サイズで規格化してからリスト形式に変換する。
        
        Parameters
        ----------
        xml_path : str
            xmlファイルへのパス。
        width : int
            対象画像の幅。
        height : int
            対象画像の高さ。
        
        Returns
        -------
        ret : [[xmin, ymin, xmax, ymax, label_ind], ... ]
            物体のアノテーションデータを格納したリスト。画像内に存在する物体数分のだけ要素を持つ。
        """
        
        # 画像内の全ての物体のアノテーションをこのリストに格納します。
        ret = []
        
        # xmlファイルを読み込む
        xml = ET.parse(xml_path).getroot()
        
        # 画像内にある物体(object)の数だけループする
        for obj in xml.iter('object'):
            
            # アノテーションで検知がdifficultに設定されているものは除外
            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                continue
                
            # 1つの物体に対するアノテーションを格納するリスト
            bndbox = []
            
            name = obj.find('name').text.lower().strip()   # 物体名
            bbox = obj.find('bndbox')   # バウンディングボックスの情報
            
            # アノテーションの xmin, ymin, xmax, ymaxを取得し、0~1に規格化
            pts = ['xmin', 'ymin', 'xmax', 'ymax']
            
            for pt in (pts):
                #  VOCは原点が(1,1)なので1を引き算
                cur_pixel = int(bbox.find(pt).text) - 1
                
                # 幅、高さで規格化
                if pt == 'xmin' or pt == 'xmax':   # x方向のときは幅で割算
                    cur_pixel /= width
                else:
                    cur_pixel /= height
                
                bndbox.append(cur_pixel)
            
            # アノテーションのクラス名のindexを取得して追加
            label_idx = self.classes.index(name)
            bndbox.append(label_idx)
            
            # resに[xmin, ymin, xmax, ymax, label_ind]を足す
            ret += [bndbox]
            
        return np.array(ret)   # [[xmin, ymin, xmax, ymax, label_ind], ...]

In [9]:
# 動作確認
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']

transform_anno = Anno_xml2list(voc_classes)

# 画像の読み込み OpenCVを使用
ind = 1
image_file_path = val_img_list[ind]  # indは例として1を選んでいるだけ
img = cv2.imread(image_file_path)  # [高さ][幅][色BGR]
height, width, channels = img.shape

# アノテーションをリストで表示
transform_anno(val_anno_list[ind], width, height)

array([[ 0.09      ,  0.03003003,  0.998     ,  0.996997  , 18.        ],
       [ 0.122     ,  0.56756757,  0.164     ,  0.72672673, 14.        ]])